In [2]:
"""Backend supported: tensorflow.compat.v1, tensorflow, pytorch, paddle"""
import deepxde as dde
import numpy as np

from consts import *

def find_region(value):
    r1 = consts["r1"]
    r2 = consts["r2"]
    r3 = consts["r3"]
    r4 = consts["r4"]

    # 判断值落在哪个区间
    if 0 <= value < r1: 
        return int(0)
    elif r1 <= value < r2:
        return int(1)
    elif r2 <= value < r3:
        return int(2)
    elif r3 <= value <= r4:
        return int(3)
    else:
        raise ValueError("Value {} is less than 0 or larger than r4".format(value))


In [5]:

def pde(x, y):
    # region_idx = find_region(x)
    Params = consts["Params"][1]
    mu = Params["mu"]
    Jz = Params["Jz"]
    dA_dr = dde.grad.jacobian(y, x)
    drdA_ddr =dde.grad.jacobian(x * dA_dr, x) 
    return drdA_ddr / (x*mu) + Jz 


def boundary_l(x, on_boundary):
    return on_boundary and dde.utils.isclose(x[0], -1)


def boundary_r(x, on_boundary):
    return on_boundary and dde.utils.isclose(x[0], 1)


def func(x):
    return (x + 1) ** 2


geom = dde.geometry.Interval(, 1)
bc_l = dde.icbc.DirichletBC(geom, func, boundary_l)
bc_r = dde.icbc.NeumannBC(geom, lambda X: 2 * (X + 1), boundary_r)
data = dde.data.PDE(geom, pde, [bc_l, bc_r], 16, 2, solution=func, num_test=100)

layer_size = [1] + [50] * 3 + [1]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn.FNN(layer_size, activation, initializer)

model = dde.Model(data, net)
model.compile("adam", lr=0.001, metrics=["l2 relative error"])
losshistory, train_state = model.train(iterations=10000)

# dde.saveplot(losshistory, train_state, issave=True, isplot=True)

Compiling model...
'compile' took 0.000478 s

Training model...

Step      Train loss                        Test loss                         Test metric   
0         [inf, 1.80e-04, 1.58e+01]         [7.53e+12, 1.80e-04, 1.58e+01]    [9.98e-01]    
1000      [nan, nan, nan]                   [nan, nan, nan]                   [nan]         

Best model at step 0:
  train loss: inf
  test loss: inf
  test metric: 

'train' took 1.213775 s

